

<center>
    <h1> INF335 - Tecnologías de Búsqueda Web   </h1>
    <h2> Tarea 1 </h2>
    <h3> Universidad Técnica Federico Santa Maria </h3>
    
</center>

_Marzo 2017_
<p>Profesor: Marcelo Mendoza</p>
 <p>Ayudante: Daniel Rivera</p>


<h2> Objetivos </h2>
<ul>
<li  style ="margin: 12px 0px; font-size:16px"> Implementar y analizar la herramienta de python NLTK (Natural Language Tookit) para trabajar con procesamiento de texto y lenguaje natural. </li>
<li style ="margin: 12px 0px;font-size:16px " > Estudiar e implementar las estructuras de datos adecuadas para representar un corpus, documentos y palabras con su categorización correspondiente.</li>
</ul>

<h2>Dataset : Amazon Fine Food Review</h2>

<p style="font-size:16px"> Para esta tarea se va a trabajar con el dataset de <i>“Amazon Fine Food Review”</i> el cual contiene más de 500.000 críticas de platos de comida y restaurants provenientes de Amazon. El archivo consiste en un .csv (“Comma Separate Values”) el cual contiene la siguiente estructura:</p>

<ol>
<li style ="margin: 5px 0px;"> <strong>Id</strong> - Id único de cada reseña</li>
<li style ="margin: 8px 0px;"><strong> ProductId</strong> - Id único que identifica el producto a analizar</li>
<li style ="margin: 8px 0px;"><strong>UserId</strong> - Id único que identifica al usuario</li>
<li style ="margin: 8px 0px;" ><strong>ProfileName</strong> - Nombre del usuario que realizó la reseña</li>
<li style ="margin: 8px 0px;"><strong>HelpfulnessNumerator</strong> -  número de usuarios que indicaron que encontraron esta crítica util</li>
<li style ="margin: 8px 0px;" ><strong>HelpfulnessDenominator</strong> número de usuarios que indicaron que encontraron esta crítica util -</li>
<li style ="margin: 8px 0px;" ><strong>Score</strong> - Rating, con valores entre 1 y 5 estrellas</li>
<li style ="margin: 8px 0px;" ><strong>Time</strong> - timestamp for the review</li>
<li style ="margin: 8px 0px;" ><strong>Summary</strong> - breve resumen de la reseña</li>
<li style ="margin: 8px 0px;"><strong>Text</strong> - string que contiene la reseña </li>
</ol>


<p style="font-size:16px" > <strong> Link Descarga Dataset: https://drive.google.com/open?id=0B1GNvIDVzwwLR2dwQVliRnBWMnM </strong>  </p> 

<p style="font-size:16px" > <b> Objetivo: </b> Extraiga del documento el item “Text” y generé un corpus , almacenando en un string todas las reseñas del dataset . Usará esta variable para realizar las siguientes etapas de preprocesamiento de texto.</p>

In [1]:
import nltk

In [20]:
import csv

with open('amazon-fine-foods/Reviews.csv', encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile)
    reseña = []
    for row in reader:
        reseña.append(row['Text'])

In [8]:
#reseña[524000]

<h2>Preprocesamiento:</h2>

<ol >
<li style ="margin: 12px 0px;" >
<p style="font-size:16px" > Si observa el corpus, se dará cuenta de que hay etiquetas <i>html</i> embebidas en algunas reseñas. Para eliminar estas etiquetas , use la libreria <i>Beautiful Soup</i> (link: https://www.crummy.com/software/BeautifulSoup/bs4/doc/).</p> </li>

</ol>


In [21]:
from bs4 import BeautifulSoup

reseñas = []
for i in range(len(reseña)):
    markup = reseña[i]
    soup = BeautifulSoup(markup, 'lxml')
    reseña_sintag = soup.get_text()
    reseñas.append(reseña_sintag)

In [51]:
#reseñas is a list of reseñas of large 568454
reseñas[0:10]

['I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.',
 'Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".',
 'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.',
 'If you are looking f

<p> A continuación, guardaremos el estado del corpus utilizando la librería Pickle </p>

In [25]:
#Using Pickle for save de state of corpus
import pickle

pickle_out = open("list.pickle", "wb")
pickle.dump(reseñas, pickle_out)
pickle_out.close()

# Correr desde acá

In [20]:
import pickle

pickle_in = open("list.pickle", "rb")

reseñas = pickle.load(pickle_in)

<p> Luego, procederemos a dejar el corpus como una lista de sentencias </p>

In [22]:
from nltk.tokenize import sent_tokenize, word_tokenize
reseña_sent_token = []

for i in range(len((reseñas))):
    reseña_sent_token.append(sent_tokenize(reseñas[i]))

In [24]:
#reseña_sent_token is a list of list that contain a tokenize sentences
reseña_sent_token[0]

['I have bought several of the Vitality canned dog food products and have found them all to be of good quality.',
 'The product looks more like a stew than a processed meat and it smells better.',
 'My Labrador is finicky and she appreciates this product better than  most.']

2<li style ="margin: 12px 0px;">
<p style="font-size:16px"> Convierta el corpus , de modo que solo existan minúsculas (<code>lowercase</code>). </p></li>

In [25]:
corpus = []

for i in range(len(reseña_sent_token)):
    for j in range(len(reseña_sent_token[i])):
        corpus.append(reseña_sent_token[i][j])

In [28]:
len(corpus) #corpus is a list of sentences

2839271

In [29]:
#save corpus for sentimental analisys 
#Using Pickle for save de state of corpus
import pickle

pickle_out = open("corpus_sent_tokenize.pickle", "wb")
pickle.dump(corpus, pickle_out)
pickle_out.close()

## Correr desde acá para Sentymental Analysis

In [4]:
import pickle

pickle_in = open("corpus_sent_tokenize.pickle", "rb")

corpus = pickle.load(pickle_in)

In [6]:
for i in range(len(corpus)):
    corpus[i] = corpus[i].lower()

In [7]:
#Corpus is a list with all words of our sentences, not a list of list. 
len(corpus)

2839271

In [31]:
corpus_word_tokenize = []

for i in range(len(corpus)):
    corpus_word_tokenize.append(word_tokenize(corpus[i]))


<li style ="margin: 12px 0px;">
<p style="font-size:16px" > Usando la lista de stopwords ortorgada por nltk, elimine aquellas palabras que sean clasificadas como stopwords, es decir, aquellas palabras que poseen poco contexto léxico y no otorgan información relevante. </p></li>

In [32]:
from nltk.corpus import stopwords

stop_words =set(stopwords.words("english"))
stop_words.update(['.', '``', "''", '-', '--','...', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}']) 

corpus_filtered = []

for i in corpus_word_tokenize:
    for j in i:
        if j not in stop_words:
            corpus_filtered.append(j)

In [11]:
len(corpus_filtered)

23702227

In [34]:
import pickle

corpus = open("corpus_word_tokenize.pickle", "wb")
pickle.dump(corpus_filtered, corpus)
corpus.close()

## Desde acá comenzamos con el corpus ya tokenizado a nivel de word

In [35]:
import pickle

pickle_in = open("corpus_word_tokenize.pickle", "rb")

corpus = pickle.load(pickle_in)

4
<li style ="margin: 12px 0px;">
<p style="font-size:16px" > Elimine las palabras que aparezcan en el corpus con una frecuencia inferior a un umbral definido (ejemplo : inferior a 3) ( para ello, es recomendable determinar previo la frecuencia de cada término usando un diccionario). </p></li>

In [3]:
#corpus_filterbyfreq contain the words with frequency > 5 
corpus_filterbyfreq = []

#diccionario de palabras
diccionario = {}

for i in set(corpus):
    diccionario[i] = 0

for i in corpus:
    diccionario[i]+= 1

In [4]:
for i in corpus:
    if(diccionario[i]>5):
        corpus_filterbyfreq.append(i)

5
<li style ="margin: 12px 0px;">
<p style="font-size:16px" > Usando nltk, determine los Top-30 collocations mas relevantes del corpus, usando Bigramas .Implemente la función <code>BigramAssocMeasures()</code> y <code>BigramCollocationFinder.from_words()</code>. Recuerde que para este punto el corpus debe estar tokenizado. (mirar documentación). </p></li>

In [27]:
import nltk
from nltk.collocations import * 

bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(corpus_filterbyfreq, 30)
finder.apply_freq_filter(5)
print('Printing Top 30 collocations: ')
print(finder.nbest(bigram_measures.likelihood_ratio, 30))

Printing Top 30 collocations: 
[('tea', 'tea'), ('coffee', 'coffee'), ('food', 'food'), ('chips', 'chips'), ('chocolate', 'chocolate'), ('popcorn', 'popcorn'), ('dog', 'dog'), ('peanut', 'butter'), ('>', '>'), ('cat', 'cat'), ('cookies', 'cookies'), ('water', 'water'), ('salt', 'salt'), ('green', 'tea'), ('%', '%'), ('hair', 'hair'), ('coconut', 'coconut'), ('treats', 'treats'), ('cup', 'coffee'), ('cat', 'food'), ('oil', 'oil'), ('bars', 'bars'), ('sauce', 'sauce'), ('rice', 'rice'), ('gluten', 'free'), ('dog', 'food'), ('$', '$'), ('bread', 'bread'), ('cereal', 'cereal'), ('jerky', 'jerky')]


6
<li style ="margin: 12px 0px;">
<p style="font-size:16px" > Usando la libreria incorporada en nltk, implemente Stanford POS tagger para categorizar y obtener los tags de cada token del corpus usando Part-Of-Speech Tagger (POSTagger). </p></li>

In [5]:
len(corpus_filterbyfreq)
corpus_filterbyfreq[0:10]

['bought',
 'several',
 'vitality',
 'canned',
 'dog',
 'food',
 'products',
 'found',
 'good',
 'quality']

In [45]:
import os
java_path="C:/Program Files/Java/jdk1.8.0_121/bin/java.exe"
os.environ['JAVAHOME'] = java_path

nltk.internals.config_java("C:/Program Files/Java/jdk1.8.0_121/bin/")
jar = '../stanford-postagger-full-2016-10-31/stanford-postagger.jar'
model = '../stanford-postagger-full-2016-10-31/models/english-left3words-distsim.tagger'

[Found C:/Program Files/Java/jdk1.8.0_121/bin/: C:/Program Files/Java/jdk1.8.0_121/bin/java.exe]


In [46]:
from nltk.tag import StanfordPOSTagger

pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8')

tags = pos_tagger.tag(corpus_filterbyfreq[0:100])

In [47]:
tags

[('bought', 'VBD'),
 ('several', 'JJ'),
 ('vitality', 'NN'),
 ('canned', 'VBD'),
 ('dog', 'NN'),
 ('food', 'NN'),
 ('products', 'NNS'),
 ('found', 'VBD'),
 ('good', 'JJ'),
 ('quality', 'NN'),
 ('product', 'NN'),
 ('looks', 'VBZ'),
 ('like', 'IN'),
 ('stew', 'NN'),
 ('processed', 'VBN'),
 ('meat', 'NN'),
 ('smells', 'VBZ'),
 ('better', 'JJR'),
 ('labrador', 'NN'),
 ('finicky', 'JJ'),
 ('appreciates', 'VBZ'),
 ('product', 'NN'),
 ('better', 'JJR'),
 ('product', 'NN'),
 ('arrived', 'VBD'),
 ('labeled', 'VBN'),
 ('jumbo', 'JJ'),
 ('salted', 'JJ'),
 ('peanuts', 'NNS'),
 ('peanuts', 'NNS'),
 ('actually', 'RB'),
 ('small', 'JJ'),
 ('sized', 'VBN'),
 ('unsalted', 'JJ'),
 ('sure', 'JJ'),
 ('error', 'NN'),
 ('vendor', 'NN'),
 ('intended', 'VBN'),
 ('represent', 'VBP'),
 ('product', 'NN'),
 ('jumbo', 'JJ'),
 ('confection', 'NN'),
 ('around', 'IN'),
 ('centuries', 'NNS'),
 ('light', 'JJ'),
 ('pillowy', 'JJ'),
 ('citrus', 'NN'),
 ('gelatin', 'NN'),
 ('nuts', 'NNS'),
 ('case', 'NN'),
 ('filberts', '

7
<li style ="margin: 12px 0px;">
<p style="font-size:16px" > Usando la libreria incorporada en nltk, implemente Named Entity Recognition (NER) con Stanford NER Tagger. Analice y describa sus resultados. </p></li>

In [48]:
from nltk.tag import StanfordNERTagger

st = StanfordNERTagger('../stanford-ner-2016-10-31/classifiers/english.all.3class.distsim.crf.ser.gz', '../stanford-ner-2016-10-31/stanford-ner.jar', encoding='utf-8')

print(st.tag(corpus_filterbyfreq[0:100]))

[('bought', 'O'), ('several', 'O'), ('vitality', 'O'), ('canned', 'O'), ('dog', 'O'), ('food', 'O'), ('products', 'O'), ('found', 'O'), ('good', 'O'), ('quality', 'O'), ('product', 'O'), ('looks', 'O'), ('like', 'O'), ('stew', 'O'), ('processed', 'O'), ('meat', 'O'), ('smells', 'O'), ('better', 'O'), ('labrador', 'O'), ('finicky', 'O'), ('appreciates', 'O'), ('product', 'O'), ('better', 'O'), ('product', 'O'), ('arrived', 'O'), ('labeled', 'O'), ('jumbo', 'O'), ('salted', 'O'), ('peanuts', 'O'), ('peanuts', 'O'), ('actually', 'O'), ('small', 'O'), ('sized', 'O'), ('unsalted', 'O'), ('sure', 'O'), ('error', 'O'), ('vendor', 'O'), ('intended', 'O'), ('represent', 'O'), ('product', 'O'), ('jumbo', 'O'), ('confection', 'O'), ('around', 'O'), ('centuries', 'O'), ('light', 'O'), ('pillowy', 'O'), ('citrus', 'O'), ('gelatin', 'O'), ('nuts', 'O'), ('case', 'O'), ('filberts', 'O'), ('cut', 'O'), ('tiny', 'O'), ('squares', 'O'), ('liberally', 'O'), ('coated', 'O'), ('powdered', 'O'), ('sugar', '

8
<li style ="margin: 12px 0px;">
<p style="font-size:16px" > <strong> Sentiment Analysis </strong>: Implemente usando la libreria <i>Vader</i> incorporada en nltk para analizar la polaridad del corpus ,determinar cada documento (para ello es necesario re-estructurar el corpus como un array de documentos, o sentencias):</p>
<ol>
<li> Tokenizar el corpus a nivel de sentencia (recuerde incorporar el preprocesamiento previo).</li>
<li> Para cada sentencias (reseña) , implemente Vader para determinar la polaridad.</li>
</ol>
</li>

In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [5]:
sid = SentimentIntensityAnalyzer()
for sentence in corpus[0:100]:
    print(sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
    print()

I have bought several of the Vitality canned dog food products and have found them all to be of good quality.
compound: 0.6369, neg: 0.0, neu: 0.776, pos: 0.224, 
The product looks more like a stew than a processed meat and it smells better.
compound: 0.6901, neg: 0.0, neu: 0.659, pos: 0.341, 
My Labrador is finicky and she appreciates this product better than  most.
compound: 0.7351, neg: 0.0, neu: 0.617, pos: 0.383, 
Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted.
compound: 0.0, neg: 0.0, neu: 1.0, pos: 0.0, 
Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
compound: -0.1027, neg: 0.129, neu: 0.762, pos: 0.11, 
This is a confection that has been around a few centuries.
compound: 0.0, neg: 0.0, neu: 1.0, pos: 0.0, 
It is a light, pillowy citrus gelatin with nuts - in this case Filberts.
compound: -0.3182, neg: 0.173, neu: 0.827, pos: 0.0, 
And it is cut into tiny squares and then liberally co

<h2>Notas</h2>
<ul>
<li style ="margin: 12px 0px; font-size: 16px" >
Para varias etapas del preprocesamiento, usará diferentes librerias disponibles en Python. Se recomienda usar el instalador de paquetes <i>pip</i> ( link: https://pypi.python.org/pypi/pip ) .
</li>
<li style ="margin: 12px 0px; font-size: 16px" >
Algunos de estos pasos del preprocesamiento pueden demorar en compilar (en algunos casos sobre 45 min, dependiendo de la máquina), por lo que es recomendable ir guardando el estado del corpus su posterior uso. Para estos casos se recomienda usar la libreria <code> pickle </code> en python (link: https://docs.python.org/2/library/pickle.html ) 
</li>
</ul>



<h2>Documentación y Ejemplos</h2>
<ul>
<li style ="margin: 12px 0px;" >
Beautiful Soup :
https://www.crummy.com/software/BeautifulSoup/
</li>
<li style ="margin: 12px 0px;" >
Bigrams and Collocations:
http://www.nltk.org/howto/collocations.html
</li>
<li style ="margin: 12px 0px;" >
Stanford PoS Tagger :
http://www.nltk.org/api/nltk.tag.html#module-nltk.tag.stanford
</li>
<li>
Stanford 'Tagger' Link Download : https://nlp.stanford.edu/software/tagger.shtml#Download
</li>
<li style ="margin: 12px 0px;" >
Stanford Ner Tagger:
https://pythonprogramming.net/named-entity-recognition-stanford-ner-tagger/
</li>
<li style ="margin: 12px 0px;" >
Sentiment Analysis with Vader: 
http://www.nltk.org/howto/sentiment.html
</li>

</ul>

<h2>Instrucciones</h2>

<ol >
<li style ="margin: 12px 0px;" >
<p style="font-size:16px" > El informe debe entregarse en un archivo jupyter notebook (diferente a este) con el código  implementado y los análisis correspondientes. El informe debe subirse en la plataforma oficial de moodle en formato comprimido (.zip) con el nombre <i>tarea1_rol.zip</i></p> </li>

<li style ="margin: 12px 0px;" >
<p style="font-size:16px" > Todas las consultas serán atendidas por el canal de consultas de moodle. </i></p> </li>

<li style ="margin: 12px 0px;" >
<p style="font-size:16px" > La fecha de entrega es el dia <strong>10 de Abril</strong> . Pasada esa fecha se descontaran 20 puntos por dia. </p> </li>


</ol>